## Shared Micromobility Vehicle Trips Data Analysis
---

### Data Source and API
- AODP Dataset Access: https://data.austintexas.gov/Transportation-and-Mobility/Shared-Micromobility-Vehicle-Trips/7d8e-dm7r
- API Endpoint: https://data.austintexas.gov/resource/7d8e-dm7r.json
- API Documentation: https://dev.socrata.com/foundry/data.austintexas.gov/7d8e-dm7r

To access the dataset host, install SODA API first:
    `pip install sodapy`

### Data Extraction

- Data Provided
    - trip_id
    - device_id
    - modified_date
    - council_district_start
    - council_district_end
    - vehicle_type
    - trip_duration
    - trip_distance
    - start_time
    - end_time
    - month
    - hour
    - day_of_week
    - year
    - census_geoid_start
    - census_geoid_end

In [2]:
# Dependencies and packages
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math as math
import datetime as dt
#import seaborn as sns
import pandas as pd
#import geopandas as gpd
from sodapy import Socrata

In [16]:
# url = https://data.austintexas.gov/resource/7d8e-dm7r.json
# Data Extraction:
client = Socrata("data.austintexas.gov", None)

results = client.get("7d8e-dm7r", where="year=2019",limit=9000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [17]:
results_df.head()

,trip_id,device_id,vehicle_type,trip_duration,trip_distance,start_time,end_time,modified_date,month,hour,day_of_week,council_district_start,council_district_end,year,census_geoid_start,census_geoid_end
0,92b80c20-89a4-4c7f-8b61-81a3570a218e,4d1e7bd0-75ce-401c-a802-8dd87122f802,scooter,678,2785,2019-04-27T15:00:00.000,2019-04-27T15:15:00.000,2019-04-28T02:42:07.000,4,15,6,1,9,2019,48453001100,48453001100
1,d9aef7c5-025d-4f2c-bafe-7801f89ef9dd,fdc38815-fe35-454b-bfb9-a63a2f2c205b,scooter,713,2466,2019-04-27T15:00:00.000,2019-04-27T15:15:00.000,2019-04-28T02:42:07.000,4,15,6,9,5,2019,48453001305,48453001901
2,baec3b6e-66f1-4043-bbd5-d54087f1c2de,007f1fed-0fe6-423f-8ca7-cd67834f20da,scooter,469,1608,2019-04-27T15:15:00.000,2019-04-27T15:15:00.000,2019-04-28T02:42:07.000,4,15,6,9,9,2019,48453001100,48453001305
3,23787175-efa2-46db-9ba6-9d53f9eabd64,672b17a6-6916-4f4e-a4e4-8168d623f638,scooter,644,1326,2019-04-27T15:15:00.000,2019-04-27T15:15:00.000,2019-04-28T02:42:07.000,4,15,6,9,9,2019,48453001305,48453001100
4,8353d103-166b-4958-aa7d-07c59cc8ef5c,10e6b006-c5c5-4012-8f73-1ece7b7d19c1,scooter,419,1511,2019-04-27T15:15:00.000,2019-04-27T15:15:00.000,2019-04-28T02:42:07.000,4,15,6,3,1,2019,48453000902,48453000804


In [5]:
# Check for missing values:
results_df.count()

trip_id                   1000000
device_id                 1000000
modified_date             1000000
council_district_start     999996
council_district_end       999996
vehicle_type               999999
trip_duration              999999
trip_distance              999999
start_time                 999999
end_time                   999999
month                      999999
hour                       999999
day_of_week                999999
year                       999999
census_geoid_start         999994
census_geoid_end           999994
dtype: int64

In [6]:
# Checking and confirming the missing values:
missing_census_geoid_start = results_df["census_geoid_start"].isnull().sum()
print(f"There are {missing_census_geoid_start} missing census_geoid_start.")

There are 6 missing census_geoid_start.


In [7]:
results_df.isnull().sum()

trip_id                   0
device_id                 0
modified_date             0
council_district_start    4
council_district_end      4
vehicle_type              1
trip_duration             1
trip_distance             1
start_time                1
end_time                  1
month                     1
hour                      1
day_of_week               1
year                      1
census_geoid_start        6
census_geoid_end          6
dtype: int64

### Data Cleaning

In [8]:
clean_df = results_df.copy()

In [9]:
clean_df.columns = clean_df.columns.str.replace('_',' ').str.title()
clean_df

,Trip Id,Device Id,Modified Date,Council District Start,Council District End,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Month,Hour,Day Of Week,Year,Census Geoid Start,Census Geoid End
0,6865731e-5e4b-4819-9435-4e62e09a652d,08c8c852-257c-49ad-a14f-727bfc79626d,2018-12-13T15:36:12.000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08dc0d9c-735e-4006-b9af-60011509e469,255716b5-4a9f-4efc-9967-7a65b3521bac,2018-12-21T00:41:42.000,9,0,bicycle,11491603,1658,2018-11-30T11:00:00.000,2019-04-12T12:00:00.000,11,11,5,2018,NaN,NaN
2,aa27d854-9f27-456e-ae49-f268a9b2b533,6b90a827-60a1-47ad-94a4-d33654dbd50d,2019-04-30T06:45:17.000,9,9,scooter,170,0,2019-04-29T17:30:00.000,2019-04-29T17:30:00.000,4,17,1,2019,48453000601,48453000601
3,b7c9f133-966b-46f1-91ce-2682ce7fcf50,1e5234a3-e86b-41e1-a1ad-e98310f3a71c,2019-04-30T06:45:17.000,9,9,scooter,90,0,2019-04-29T17:30:00.000,2019-04-29T17:30:00.000,4,17,1,2019,48453000601,48453000601
4,1d4f5d89-c044-46cb-8674-80d37fa0371f,caa0325c-7c0b-4909-a573-d83126ecc953,2019-04-30T06:45:17.000,9,9,scooter,570,826,2019-04-29T17:15:00.000,2019-04-29T17:15:00.000,4,17,1,2019,48453001100,48453001100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,8ed87c2b-93d4-4719-907c-0af5628fa6b7,ee97b0fa-4e16-4394-b852-62e7b748af37,2019-08-02T12:01:45.000,9,3,scooter,940,2134,2019-08-01T23:15:00.000,2019-08-01T23:30:00.000,8,23,4,2019,48453001100,48453000902
999996,63fd1c6f-b431-484f-b631-9cd15524ea31,84e04210-7bdf-4dcc-ac4c-1b7bb8179659,2019-09-01T09:30:29.000,9,9,scooter,36,42,2019-08-31T20:45:00.000,2019-08-31T20:45:00.000,8,20,6,2019,48453000604,48453000604
999997,001ffe77-2454-4d8d-8a84-5188ab57158d,33f1d664-eeba-4057-8a7c-efcbf0c2a52e,2019-07-24T11:30:28.000,4,4,scooter,223,742,2019-07-23T22:45:00.000,2019-07-23T22:45:00.000,7,22,2,2019,48453001503,48453001503
999998,9b16ebea-2cc6-411d-a0e1-f36c1e15cbbc,a389fc29-00d3-4e94-af33-6b0034e22d1b,2019-07-24T11:30:28.000,9,9,scooter,261,821,2019-07-23T22:45:00.000,2019-07-23T23:00:00.000,7,22,2,2019,48453000603,48453000604


In [10]:
# Renaming some column names:
clean_df = clean_df.rename(columns = {
    "Trip Id": "Trip ID",
    "Device Id": "Device ID",
    "Census Geoid Start": "Census GEOID Start",
    "Census Geoid End": "Census GEOID End",
})
clean_df.head()

,Trip ID,Device ID,Modified Date,Council District Start,Council District End,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Month,Hour,Day Of Week,Year,Census GEOID Start,Census GEOID End
0,6865731e-5e4b-4819-9435-4e62e09a652d,08c8c852-257c-49ad-a14f-727bfc79626d,2018-12-13T15:36:12.000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08dc0d9c-735e-4006-b9af-60011509e469,255716b5-4a9f-4efc-9967-7a65b3521bac,2018-12-21T00:41:42.000,9,0,bicycle,11491603,1658,2018-11-30T11:00:00.000,2019-04-12T12:00:00.000,11,11,5,2018,NaN,NaN
2,aa27d854-9f27-456e-ae49-f268a9b2b533,6b90a827-60a1-47ad-94a4-d33654dbd50d,2019-04-30T06:45:17.000,9,9,scooter,170,0,2019-04-29T17:30:00.000,2019-04-29T17:30:00.000,4,17,1,2019,48453000601,48453000601
3,b7c9f133-966b-46f1-91ce-2682ce7fcf50,1e5234a3-e86b-41e1-a1ad-e98310f3a71c,2019-04-30T06:45:17.000,9,9,scooter,90,0,2019-04-29T17:30:00.000,2019-04-29T17:30:00.000,4,17,1,2019,48453000601,48453000601
4,1d4f5d89-c044-46cb-8674-80d37fa0371f,caa0325c-7c0b-4909-a573-d83126ecc953,2019-04-30T06:45:17.000,9,9,scooter,570,826,2019-04-29T17:15:00.000,2019-04-29T17:15:00.000,4,17,1,2019,48453001100,48453001100


In [11]:
# Drop all the null values
clean_df = clean_df.dropna(how='any')
clean_df.head()

,Trip ID,Device ID,Modified Date,Council District Start,Council District End,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Month,Hour,Day Of Week,Year,Census GEOID Start,Census GEOID End
2,aa27d854-9f27-456e-ae49-f268a9b2b533,6b90a827-60a1-47ad-94a4-d33654dbd50d,2019-04-30T06:45:17.000,9,9,scooter,170,0,2019-04-29T17:30:00.000,2019-04-29T17:30:00.000,4,17,1,2019,48453000601,48453000601
3,b7c9f133-966b-46f1-91ce-2682ce7fcf50,1e5234a3-e86b-41e1-a1ad-e98310f3a71c,2019-04-30T06:45:17.000,9,9,scooter,90,0,2019-04-29T17:30:00.000,2019-04-29T17:30:00.000,4,17,1,2019,48453000601,48453000601
4,1d4f5d89-c044-46cb-8674-80d37fa0371f,caa0325c-7c0b-4909-a573-d83126ecc953,2019-04-30T06:45:17.000,9,9,scooter,570,826,2019-04-29T17:15:00.000,2019-04-29T17:15:00.000,4,17,1,2019,48453001100,48453001100
5,cd442fb9-058f-46a7-af90-b4adb56fd163,1653cf10-75f5-4934-a9d9-c8d43ade67a9,2019-04-30T06:45:17.000,9,9,scooter,438,1815,2019-04-29T17:30:00.000,2019-04-29T17:45:00.000,4,17,1,2019,48453000601,48453000500
6,e1ac0e42-ab2d-4eac-b019-f085169c1d38,1d89f9e2-d01a-4736-b7d8-258cb9a2108c,2019-04-30T06:45:17.000,9,9,scooter,391,1519,2019-04-29T17:30:00.000,2019-04-29T17:30:00.000,4,17,1,2019,48453000401,48453000307


In [12]:
# Change the time and date format for columns - 'Modified Date', 'Start time' and 'End Time'
clean_df['Start Time'] = pd.to_datetime(clean_df['Start Time'])
clean_df['End Time'] = pd.to_datetime(clean_df['End Time'])
clean_df['Modified Date'] = pd.to_datetime(clean_df['Modified Date'])
clean_df.head()

,Trip ID,Device ID,Modified Date,Council District Start,Council District End,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Month,Hour,Day Of Week,Year,Census GEOID Start,Census GEOID End
2,aa27d854-9f27-456e-ae49-f268a9b2b533,6b90a827-60a1-47ad-94a4-d33654dbd50d,2019-04-30 06:45:17,9,9,scooter,170,0,2019-04-29 17:30:00,2019-04-29 17:30:00,4,17,1,2019,48453000601,48453000601
3,b7c9f133-966b-46f1-91ce-2682ce7fcf50,1e5234a3-e86b-41e1-a1ad-e98310f3a71c,2019-04-30 06:45:17,9,9,scooter,90,0,2019-04-29 17:30:00,2019-04-29 17:30:00,4,17,1,2019,48453000601,48453000601
4,1d4f5d89-c044-46cb-8674-80d37fa0371f,caa0325c-7c0b-4909-a573-d83126ecc953,2019-04-30 06:45:17,9,9,scooter,570,826,2019-04-29 17:15:00,2019-04-29 17:15:00,4,17,1,2019,48453001100,48453001100
5,cd442fb9-058f-46a7-af90-b4adb56fd163,1653cf10-75f5-4934-a9d9-c8d43ade67a9,2019-04-30 06:45:17,9,9,scooter,438,1815,2019-04-29 17:30:00,2019-04-29 17:45:00,4,17,1,2019,48453000601,48453000500
6,e1ac0e42-ab2d-4eac-b019-f085169c1d38,1d89f9e2-d01a-4736-b7d8-258cb9a2108c,2019-04-30 06:45:17,9,9,scooter,391,1519,2019-04-29 17:30:00,2019-04-29 17:30:00,4,17,1,2019,48453000401,48453000307


In [13]:
#Find the total number of scooter rides:
scooters = clean_df[clean_df["Vehicle Type"] == "scooter"]
total_scooters = len(clean_df) - scooters.shape[0]
total_scooters

68153

In [14]:
#Find the total number of bicycle rides:
bicycles = clean_df[clean_df["Vehicle Type"] == "bicycle"]
total_bicycles = len(clean_df) - bicycles.shape[0]
total_bicycles

931841

In [15]:
#Check to tally the total rides 
total_rides = total_bicycles + total_scooters
total_rides

999994

In [ ]:
#????????

# Does the distance of the trip impact the method of transportation (scooter vs. bicycle)
    #Type of ride; distance per type of rides, average for each
    #Plot this

#Total rides = 999994
#Bicycles = 931841

#scooter_distance = clean_df.mean()[scooters["Trip Duration"]]
#scooter_distance


#avg_scooter_distance = 

#bike_distance = 
#avg_bike_distance =

In [ ]:
# Find the number of unique devices
#device_id_list =  clean_df["Device ID"].value_counts()
#device_id_list

In [ ]:
# Find the number of unique GEOIDs where the trips started
start_geoid = clean_df["Census GEOID Start"].value_counts()
start_geoid

In [ ]:
# Find the number of unique GEOIDs where the trips ended
end_geoid = clean_df["Census GEOID End"].value_counts()
end_geoid

In [ ]:
# Convert dataframe to CSV 
output_data_file = "/Users/sheetalbongale/Scooters_In_Austin_Data_Analysis/resources/shared_mobility_data.csv"
clean_df.to_csv(output_data_file)

### Data Merging

In [ ]:
csvpath1 = os.path.join("/Users/sheetalbongale/Scooters_In_Austin_Data_Analysis/resources/zip_tract_092019.csv")
csvpath2 = os.path.join("/Users/sheetalbongale/Scooters_In_Austin_Data_Analysis/resources/shared_mobility_data.csv")
zip_data_start = pd.read_csv(csvpath1)
zip_data_end = pd.read_csv(csvpath1)
clean_df = pd.read_csv(csvpath2, low_memory=False)

In [ ]:
clean_df.head(10)

In [ ]:
clean_df = clean_df.drop(columns=['Unnamed: 0'])
clean_df

In [ ]:
#column3 = zip_data['tract']
#zip_data['Census GEOID End'] = column3
zip_data_start.rename(columns={'zip': 'Zipcode', 'tract': 'Census GEOID Start'}, inplace=True)
zip_data_start

In [ ]:
zip_data_end.rename(columns={'zip': 'Zipcode', 'tract': 'Census GEOID End'}, inplace=True)
zip_data_end

In [ ]:
zip_data_start[["Zipcode", "Census GEOID Start"]] = zip_data_start[["Zipcode", "Census GEOID Start"]].astype(int).astype(str)
zip_data_end[["Zipcode", "Census GEOID End"]] = zip_data_end[["Zipcode", "Census GEOID End"]].astype(int).astype(str)


In [ ]:
df_start = pd.merge(clean_df, zip_data_start, on= "Census GEOID Start", how ="right")
df_start

In [ ]:
df_start = df_start.rename(columns={'Zipcode': 'Zipcode Start'})

In [ ]:
df_start = df_start.dropna()

In [ ]:
df_start.dtypes

In [ ]:
zip_data_start.dtypes

In [ ]:
df_end = pd.merge(df_start, zip_data_end, on= "Census GEOID End", how ="right")
df_end

In [ ]:
df_end = df_end.rename(columns={'Zipcode': 'Zipcode End'})

In [ ]:
df_end = df_end.dropna()
df_end

In [ ]:
df = df_end.copy()

In [ ]:
df.isnull().sum()

In [ ]:
#Commenting this out because we already have the csv file after running this once
#output_data_file = "/Users/sheetalbongale/Scooters_In_Austin_Data_Analysis/resources/merged_sharedmobility.csv"
#df.to_csv(output_data_file)

In [ ]:
# csvpath = os.path.join("/Users/sheetalbongale/Scooters_In_Austin_Data_Analysis/resources/merged_sharedmobility.csv")
# df = pd.read_csv(csvpath, index_col=0, low_memory=False)

In [ ]:
df.head(30)

### Visualization of the Clean Data Set

In [ ]:
# Count trips per day and sort by day:
daily_total = pd.DataFrame(df['Day Of Week'].value_counts().sort_index())

# Map day of week with actual names:
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
daily_total['Day'] = days

# Plot glyph: 
daily_total.plot(kind='bar', x='Day', y='Day Of Week', title='Total Trip Counts by Day of week', figsize = (10,5), rot= 30, legend=False)
plt.ylabel("Number of Trips")
plt.savefig("Plots/trips_per_week.png")
plt.show()

In [ ]:
# Count trips per hour and sort by hour:
hourly_total = pd.DataFrame(df['Hour'].value_counts().sort_index())
#hours = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18', '19','20','21','22','23']
#hours = ['0','1','10','11','12','13','14','15','16','17','18','19','2','20','21','22','23','3','4','5','6','7','8','9']
#hourly_total['Hours'] = hours

hourly_total.reset_index().plot(kind='bar', x='index', y='Hour', title='Total Trip Counts by Hour', figsize = (10,5), legend=False)
plt.xlabel('Hours')
plt.ylabel('Number of Trips')
plt.savefig("Plots/trips_per_hour.png")
plt.show()
#now shows correct plot with sorted xticks

In [ ]:
monthly_total = pd.DataFrame(df['Month'].value_counts()).sort_index()
import calendar
mn=[calendar.month_name[int(x)] for x in monthly_total.index.values.tolist()]

monthly_chart = monthly_total.plot.bar(title = "Total Trips per Month ",width = 0.75,figsize = (10,5),rot = 30, legend = False)
monthly_chart.set_xticklabels(mn)
monthly_chart.set_xlabel("Trip Months")
monthly_chart.set_ylabel("Total Trip Count")
plt.savefig("Plots/trips_per_month1.png")
plt.show()
#now shows correct plot with sorted months

In [ ]:
month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
monthly_total['Months'] = month_list
monthly_total.reset_index().plot(kind='bar', x='Months', y='Month', title='Total Trip Counts by months', figsize = (10,5), rot = 30, legend=False)
plt.ylabel('Number of Trips')
plt.savefig("Plots/trips_per_month2.png")
plt.show()
#now shows correct plot with sorted xticks

In [ ]:
# Count how many trips started in each census GEOID tract
census_trip_start = pd.DataFrame(df['Census GEOID Start'].value_counts())

# Count how many trips ended in each census tract
census_trip_end = pd.DataFrame(df['Census GEOID End'].value_counts())

plt.bar(df["Census GEOID Start"], census_trip_start, figsize = (10,5))
plt.title("Total Count of trips starting per Census Tract")
plt.xlabel("GEOID ID")
plt.ylabel("Total Trips")
plt.show()